In [1]:
# %pip install -r requirements.txt

In [2]:
from langchain.document_loaders import TextLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [3]:
# 2. Load your business proposal
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data\Sakina BP.pdf") 
documents = loader.load()

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Joy Anne\AppData\Local\Temp\ipykernel_22644\225404614.py:3: SyntaxWarning: invalid escape sequence '\S'
  loader = PyPDFLoader("data\Sakina BP.pdf")


In [4]:
# 3. Split text into chunks
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=500, separator='\n')
docs = splitter.split_documents(documents)

In [5]:
# 4. Generate embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Lightweight

C:\Users\Joy Anne\AppData\Local\Temp\ipykernel_22644\1061242017.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Lightweight
c:\Users\Joy Anne\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 5. Store in Chroma DB
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embedding_model)

In [7]:
# 6. Setup retriever
retriever = vectorstore.as_retriever()

In [8]:
# 7. Setup LLM for generation
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2",
    tokenizer="deepset/roberta-base-squad2"
)

Device set to use cpu


In [ ]:
# 8. Retrieve 
query = "What is Sakina?"
retrieved_docs = retriever.get_relevant_documents(query)
context = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs[:2]])
print(context)

C:\Users\Joy Anne\AppData\Local\Temp\ipykernel_22644\3500329325.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [12]:
# Generate 
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

As a friendly customer support, answer the question based on the above context: {query}
"""

from langchain_core.prompts import ChatPromptTemplate
propmpt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = propmpt_template.format(context=context, query=query)
print(prompt)

Human: 
Answer the question based only on the following context:

5) Impact on Mental Health Research & Innovation:  With a well -organized database, 
Sakina will drive mental health research and discoveries, contributing to evidence -based 
practices and the development of new, innovative mental health solutions. The more data 
we gather, the better we can understand and im prove mental health care. Sakina will 
provide insights into trends, treatment efficacy, and patient outcomes, enabling continuous 
improvement in services. 
 
Business Model: 
Sakina operates under a sustainable and scalable revenue model that ensures accessibility for 
clinics and hospitals of all sizes: 
B2B Partnerships:  Subscription-based services tailored for hospitals and clinics, with 
tiered pricing to accommodate different institution needs. 
 
Open Access Resources: Free educational resources to increase mental health awareness 
and promote community engagement. 
 
Premium Features: Advanced analytics, 

In [16]:
# 8. Use local model to answer
result = qa_pipeline(question=prompt)
print("Answer:", result['answer'])

ValueError: Unknown arguments {'question': 'Human: \nAnswer the question based only on the following context:\n\n5) Impact on Mental Health Research & Innovation:  With a well -organized database, \nSakina will drive mental health research and discoveries, contributing to evidence -based \npractices and the development of new, innovative mental health solutions. The more data \nwe gather, the better we can understand and im prove mental health care. Sakina will \nprovide insights into trends, treatment efficacy, and patient outcomes, enabling continuous \nimprovement in services. \n \nBusiness Model: \nSakina operates under a sustainable and scalable revenue model that ensures accessibility for \nclinics and hospitals of all sizes: \nB2B Partnerships:  Subscription-based services tailored for hospitals and clinics, with \ntiered pricing to accommodate different institution needs. \n \nOpen Access Resources: Free educational resources to increase mental health awareness \nand promote co